In [1]:
from langkit import llm_metrics, extract
import re
import json
import time
import pandas as pd
headers_openllm = {"Content-Type": "application/json", "Accept": "application/json","AUTHORIZATION": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"}
url_openllm = 'https://refract.fosfor.com/vllm/mistral/v1/completions'

[nltk_data] Downloading package vader_lexicon to /home/mosaic-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/pip_packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-03-06 09:57:23,146] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [1]:
!pip install langkit[all]

  Using cached https://files.pythonhosted.org/packages/1a/e6/c7e550831ee412cad8b7abc6b664cd6e37b67acd40f0824010b07606e2ce/langkit-0.0.31-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/7f/5b047effafbdd34e52c9e2d7e44f729a0655efafb22198c45cf692cdc157/pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d8/33/302083f47386d651e4b42923f5206eeb9ee0545ea94bb506324d05fd2274/textstat-0.7.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e2/4f/58919358958405bea12f3dbf3b0567b48915998f2bf3309ba95cfe53fde0/whylogs-1.3.25-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl
  Using cached https://files.pythonhosted

  Using cached https://files.pythonhosted.org/packages/ad/80/8fc9a4d76b259c901f2c85ed10f330a8fb51993a577bddfd53a852595e12/xxhash-3.4.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/eb/57f1f43f60aa3a21296171d353b6597c312b45d9a5addb1fb5313ec3611a/sentencepiece-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b6/4d/fbe6d89fde59d8107f0a02816c4ac4542a8f9a85559fdf33c68282affcc1/transformers-4.38.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/f3/2b3a6dc5986303b3dd1bbbcf482022acb2583c428cd23f0b6d37b1a1a519/responses-0.18.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d0/de/3f37b2568115c7ebeae39508dc1092f04f3dc286f22ef30171baca9c9cf2/tldextract-5.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/f8/146b7ef0396f6c23bc014247a5e9440e113283a8078b3f806bac66ef10f1/spacy-3.7.4-cp38

  Using cached https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c96257acb4fce61e7de14/certifi-2024.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3d/09/d82fe4a34c5f0585f9ea1df090e2a71eb9bb1e469723053e1ee9f57c16f3/charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/61/a3/c307d4af64e695d13e8587d3f996a51b134156c0e8e2e26f4135bb2bf517/multidict-6.0.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3

  Using cached https://files.pythonhosted.org/packages/4c/1c/ff6546b6c12603d8dd1070aa3c3d273ad4c07f5771689a7b69a550e8c951/backcall-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9a/41/220f49aaea88bc6fa6cba8d05ecf24676326156c23b991e80b3f2fc24c77/pickleshare-0.7.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9d/42/d233ec75bc8682e368e96073703f01dc095792019fcb541e741b28ce4074/blis-0.7.11-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/39/78/f9d18da7b979a2e6007bfcea2f3c8cc02ed210538ae1ce7e69092aed7b18/confection-0.1.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/6e/45b57a7d4573d85d0b0a39d99673dc1f5eea9d92a1a4603b35e968fbf89a/cloudpathlib-0.16.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/95/04/ff642e65ad6b90db43e668d70ffb6736436c7ce41fcc549f4e9472234127/h11-0.14.0-py3-none-any.whl
  Using cached https://fi

In [11]:
def narrative_generation_with_openllm(intermediate_result, question):
  
  data_openllm = {"prompt": prompt,
                  "max_tokens": 200,
                  "temperature": 0.01,
                  "model": "/llmmodels/NLG_FINETUNEDMODELS/DATASET_MODELS/FINETUNED_MISTRALV2_V5/MISTRAL_MERGED_MODEL_JAN30_V3_0021",
                  "stop": "["}
  start = time.time()
  response = requests.post(url_openllm, json=data_openllm, headers=headers_openllm)
  end = time.time()
  open_llm_narrative_json = json.loads(response.text)
  open_llm_narrative = re.search(r'\n([^|\n]*)', open_llm_narrative_json["choices"][0]["text"]).group(1)
  open_llm_response_time = str(end - start) + str(" seconds")
  output_dict = {}
  output_dict = {
    "opensource_llm_narrative":open_llm_narrative,
    "opensource_llm_response_time":open_llm_response_time
  }
  return output_dict

In [17]:
question = input("Enter the question")
intermediate_result = input("Enter the intermediate_result")
prompt = """Given the input :\n"""+str(intermediate_result)+"""\nAnd the query: """+ str(question)+"""\n is converted into below narrative.\n[Narrative]"""
#Prompt validation
data = [[question,""]]
prompt_response = pd.DataFrame(data, columns=['prompt', 'response'])
metrics_df = extract(prompt_response)

print("\n\nPrompt Metrices")

print("toxicity level of prompt:" , metrics_df['prompt.toxicity'].values[0])
print("Patterns in prompt:" , metrics_df['prompt.has_patterns'].values[0])
print("Sentiment level of prompt:" , metrics_df['prompt.sentiment_nltk'].values[0])
print("flesch_reading_ease of prompt:" , metrics_df['prompt.flesch_reading_ease'].values[0])
print("automated_readability_index of prompt:" , metrics_df['prompt.automated_readability_index'].values[0])
print("aggregate_reading_level of prompt:" , metrics_df['prompt.aggregate_reading_level'].values[0])
print("syllable_count of prompt:" , metrics_df['prompt.syllable_count'].values[0])
print("lexicon_count of prompt:" , metrics_df['prompt.lexicon_count'].values[0])
print("sentence_count of prompt:" , metrics_df['prompt.sentence_count'].values[0])
print("character_count of prompt:" , metrics_df['prompt.character_count'].values[0])
print("polysyllable_count of prompt:" , metrics_df['prompt.polysyllable_count'].values[0])
print("monosyllable_count of prompt:" , metrics_df['prompt.monosyllable_count'].values[0])
print("difficult_words of prompt:" , metrics_df['prompt.difficult_words'].values[0])
print("jailbreak_similarity of prompt:" , metrics_df['prompt.jailbreak_similarity'].values[0])

output_dict = narrative_generation_with_openllm(intermediate_result,question)


print("\n",output_dict['opensource_llm_narrative'],"\n")

data = [[question,output_dict['opensource_llm_narrative']]]
prompt_response = pd.DataFrame(data, columns=['prompt', 'response'])
metrics_df = extract(prompt_response)

print("\n\nResponse Metrices")
print("Time taken" , output_dict["opensource_llm_response_time"])
print("toxicity level of response:" , metrics_df['response.toxicity'].values[0])
print("Patterns in response:" , metrics_df['response.has_patterns'].values[0])
print("Sentiment level of response:" , metrics_df['response.sentiment_nltk'].values[0])
print("flesch_reading_ease of response:" , metrics_df['response.flesch_reading_ease'].values[0])
print("automated_readability_index of response:" , metrics_df['response.automated_readability_index'].values[0])
print("aggregate_reading_level of response:" , metrics_df['response.aggregate_reading_level'].values[0])
print("syllable_count of response:" , metrics_df['response.syllable_count'].values[0])
print("lexicon_count of response:" , metrics_df['response.lexicon_count'].values[0])
print("sentence_count of response:" , metrics_df['response.sentence_count'].values[0])
print("character_count of response:" , metrics_df['response.character_count'].values[0])
print("polysyllable_count of response:" , metrics_df['response.polysyllable_count'].values[0])
print("monosyllable_count of response:" , metrics_df['response.monosyllable_count'].values[0])
print("difficult_words of response:" , metrics_df['response.difficult_words'].values[0])
print("refusal_similarity of response:" , metrics_df['response.refusal_similarity'].values[0])
print("relevance_to_prompt of response:" , metrics_df['response.relevance_to_prompt'].values[0])

Enter the questionsales of chair in india across months
Enter the intermediate_result{'Total': 68874, 'Minimum_Value': 61.62, 'Maximum_Value': 4572.87, 'Average': 1680, 'Difference': 4511.25, 'Stable_Case': False, 'Last_Period_Zero': False, 'Label_Available': False, 'Trailing_Zeroes': False, 'Mixed_Data': False, 'Delta': False, 'Delta_Overlap': False, 'Delta_Change_Overlap': False, 'Yoy_Available': True, 'Maximum_Date': '2023-06-01', 'Minimum_Date': '2021-04-01', 'Last_Date': ['2023-08-01', '2023-09-01'], 'Last_Values': [3279.18, 1289.61], 'Interval_Label': 'MoM', 'Date_Interval': 'month', 'Final_Change_Percentage': -61, 'Final_Period_Change_Note': 'not_good', 'Direction': 'up', 'Maximum_Growth_Percentage': 1732.99, 'Maximum_Growth_From': 116.04, 'Maximum_Growth_To': 2127.0, 'Maximum_Growth_Date_From': '2020-10-01', 'Maximum_Growth_Date_To': '2020-11-01', 'Maximum_Decline_Percentage': -95.93, 'Maximum_Decline_From': 1514.4, 'Maximum_Decline_To': 61.62, 'Maximum_Decline_Date_From': '202

/tmp/pip_packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/tmp/pip_packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




Prompt Metrices
toxicity level of prompt: 0.0008220076560974121
Patterns in prompt: None
Sentiment level of prompt: 0.0
flesch_reading_ease of prompt: 106.67
automated_readability_index of prompt: 2.9
aggregate_reading_level of prompt: 3.0
syllable_count of prompt: 8
lexicon_count of prompt: 7
sentence_count of prompt: 1
character_count of prompt: 31
polysyllable_count of prompt: 0
monosyllable_count of prompt: 6
difficult_words of prompt: 1
jailbreak_similarity of prompt: 0.04716765135526657

 For India and Chairs, Sales stands at 1.29K, decrease of 61% (MoM) in the last month(+19.07% YoY).The highest single period climb of 1.73K% (MoM) was seen in November 2020. 



Response Metrices
Time taken 4.940428733825684 seconds
toxicity level of response: 0.0007417798042297363
Patterns in response: None
Sentiment level of response: 0.0
flesch_reading_ease of response: 89.45
automated_readability_index of response: 5.6
aggregate_reading_level of response: 6.0
syllable_count of response: 38


/tmp/pip_packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/tmp/pip_packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [18]:
import os
os.environ["OPENAI_API_KEY"] = ""

from langkit import response_hallucination
from langkit.openai import OpenAILegacy


response_hallucination.init(llm=OpenAILegacy(model="gpt-3.5-turbo-instruct"), num_samples=1)

result = response_hallucination.consistency_check(
    prompt="Who was Philip Hayworth?",
    response="Philip Hayworth was an English barrister and politician who served as Member of Parliament for Thetford from 1859 to 1868.",
)

[nltk_data] Downloading package punkt to /home/mosaic-ai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Exception: Response Hallucination - Error generating sample: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable